In [ ]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

import datautils

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 5.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [ ]:
num_classes = 11
feature_dim = datautils.FEATURE_DIM
ivector_dim = 800

In [ ]:
print("Loading GloVe")
glove_dict = datautils.load_glove_dict()
print("Loading training data")
X_train, lens_train = datautils.load_train_examples(glove_dict)
ivectors_train = datautils.load_train_ivectors()
y_train = datautils.load_train_labels()
print("Loading dev data")
X_val, lens_val = datautils.load_dev_examples(glove_dict)
y_val = datautils.load_dev_labels()
ivectors_val = datautils.load_dev_ivectors()

In [ ]:
# Center and normalize the ivectors
ivectors_train -= np.mean(ivectors_train, axis=0)
ivectors_train /= np.std(ivectors_train, axis=0)

ivectors_val -= np.mean(ivectors_val, axis=0)
ivectors_val /= np.std(ivectors_val, axis=0)

print(np.mean(ivectors_train), np.std(ivectors_train))
print(np.mean(ivectors_val), np.std(ivectors_val))

In [ ]:
print("X_train:", X_train.shape)
print("ivectors_train:", ivectors_train.shape)
print("lens_train:", lens_train.shape)
print("y_train:", y_train.shape)

print("X_val:", X_val.shape)
print("ivectors_val:", ivectors_val.shape)
print("lens_val:", lens_val.shape)
print("y_val:", y_val.shape)

In [ ]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, None, feature_dim])
y = tf.placeholder(tf.int64, [None])
ivectors = tf.placeholder(tf.float32, [None, ivector_dim])
lens = tf.placeholder(tf.int64, [None])

lr = 1e-2
dr = 0.2

learning_rate = tf.placeholder(tf.float32)
dropout = tf.placeholder(tf.float32)
is_training = tf.placeholder(tf.bool)

cell_size = 192
cell1 = tf.contrib.rnn.GRUCell(cell_size)
cell2 = tf.contrib.rnn.GRUCell(cell_size)
multicell = tf.contrib.rnn.MultiRNNCell(cells=[cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multicell, X, dtype=tf.float32, sequence_length=lens) # Outputs: N x L x W
#sampled_outputs = tf.reshape(outputs[:, 3:211:3], [-1, 70 * cell_size])
batch_size = tf.shape(X)[0]
indices = tf.stack([tf.range(batch_size), tf.maximum(0, tf.cast(lens, tf.int32) - 1)], axis = 1)
sampled_outputs = tf.gather_nd(outputs, indices)

h1 = tf.concat((sampled_outputs, ivectors), axis=1)

d1 = tf.layers.dropout(h1, rate=dropout, training=is_training)
fc1 = tf.layers.dense(d1, 256, activation=tf.nn.relu)

d2 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
y_out = tf.layers.dense(d2, num_classes)

mean_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, num_classes), logits=y_out))
optimizer = tf.train.AdamOptimizer(learning_rate)

# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

In [ ]:
def run_model(session, predict, loss_val, Xd, yd, lensd, ivectorsd,
              epochs=1, batch=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch))):
            # generate indicies for the batch
            start_idx = (i*batch)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch]
            
            # get batch size
            actual_batch_size = yd[i:i+batch].shape[0]

            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx],
                         y: yd[idx],
                         lens: lensd[idx],
                         ivectors: ivectorsd[idx],
                         is_training: training_now,
                         learning_rate: lr,
                         dropout: dr
                         }
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            iter_cnt += 1
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))            
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
train_losses = []
train_acc = []
val_losses = []
val_acc = []

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, "gru-model-11")
with open("gru-model.txt") as logs:
    train_losses = [ float(val) for val in logs.readline().split() ]
    val_losses = [ float(val) for val in logs.readline().split() ]
    train_acc = [ float(val) for val in logs.readline().split() ]
    val_acc = [ float(val) for val in logs.readline().split() ]

In [ ]:
epochs = 10
lr = 1e-4
dr = 0.2
for i in range(epochs):
    loss, acc = run_model(session=sess,
                          predict=y_out,
                          loss_val=mean_loss,
                          Xd=X_train,
                          yd=y_train,
                          lensd=lens_train,
                          ivectorsd=ivectors_train,
                          epochs=1,
                          batch=128,
                          print_every=25,
                          training=train_step,
                          plot_losses=True)
    train_losses.append(loss)
    train_acc.append(acc)
    loss, acc = run_model(session=sess,
                          predict=y_out,
                          loss_val=mean_loss,
                          Xd=X_val,
                          yd=y_val,
                          lensd=lens_val,
                          ivectorsd=ivectors_val,
                          epochs=1,
                          batch=X_val.shape[0],
                          print_every=1,
                          training=None,
                          plot_losses=False)
    val_losses.append(loss)
    val_acc.append(acc)
    saver = tf.train.Saver()
    saver.save(sess, 'fusion-model', global_step=len(val_acc))

In [ ]:
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Val')
plt.grid(True)
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(train_acc, label='Train')
plt.plot(val_acc, label='Val')
plt.grid(True)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
f = open("gru-model.txt", 'w')
f.write(" ".join([str(num) for num in train_losses]) + "\n")
f.write(" ".join([str(num) for num in val_losses]) + "\n")
f.write(" ".join([str(num) for num in train_acc]) + "\n")
f.write(" ".join([str(num) for num in val_acc]) + "\n")
f.close()

In [ ]:
model_name = "fusion-model"

best_epoch = np.argmax(val_acc) + 1

saver = tf.train.Saver(max_to_keep=None)
saver.restore(sess, "./{}-{}".format(model_name, best_epoch))
print("Restoring {}-{}".format(model_name, best_epoch))

In [ ]:
predictions = tf.argmax(y_out, axis=1)
feed_dict = {X: X_val,
             lens: lens_val,
             ivectors : ivectors_val,
             y: y_val,
             is_training: False,
             dropout: 0.2,
             learning_rate: 1e-3
            }
out = sess.run([predictions], feed_dict=feed_dict)[0]

In [ ]:
class_accs = np.zeros(11)
for i in range(1100):
    class_accs[y_val[i]] += 1 if y_val[i] == out[i] else 0
print("\n".join([str(int(acc)) for acc in class_accs]))
print(np.mean(y_val == out))